<a href="https://colab.research.google.com/github/suryasuresh91/Lexical-Complexity-Prediction/blob/main/Features_Single.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy-syllables
!python -m spacy download en_core_web_sm
!pip3 install wordfreq

     |████████████████████████████████| 6.0 MB 4.3 MB/s 
     |████████████████████████████████| 1.9 MB 41.8 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 628 kB 65.6 MB/s 
     |████████████████████████████████| 181 kB 65.4 MB/s 
     |████████████████████████████████| 10.1 MB 22.5 MB/s 
     |████████████████████████████████| 451 kB 42.3 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled 

In [2]:
import numpy as np
from wordfreq import word_frequency


In [3]:
# Construct features for single word expressions

def prepare_features_single_word(tokens, sentences, nlp, word_to_ix, model, embedding_index, EMBEDDING_DIM):
  features = []
  for idx, word in enumerate(tokens):
    word = word.lower()
    feature = []

    # Word length
    feature.append(len(word))
    doc = nlp(word)

    # Syllable count and word frequency in the corpus
    # Spacy tokenizes the input sentence
    # In this case we would have only one token, the target word
    for token in doc:
      feature.append(token._.syllables_count)
      feature.append(word_frequency(word, 'en'))

    # Probability of target word `word` in the sentence estimated from by `model`
    if word in word_to_ix:
      # Output scores for each of the word in the sentence
      out = model(sentences[idx])
      pos = -1
      for itr, token in enumerate(sentences[idx].split()):
        if token.lower() == word:
          pos = itr
          break
      id_pos = word_to_ix[word] # word to id mapping
      feature.append(float(out[pos][id_pos]))
    else:
      # `word` not in vocabulary, so cannot predict probability in context
      feature.append(0.0)

    # GloVE embedding for the `word`
    if word in embedding_index:
      feature.extend(embedding_index[word].tolist())
    else:
      # `word` not in the GloVE corpus, take a random embedding
      feature.extend(np.random.random(EMBEDDING_DIM).tolist())
    features.append(feature)

    if (idx + 1) % 500 == 0:
      print('Prepared features for {} single target word sentences'.format(idx + 1))
  return features
